# 감성분석 


## Graph Attention Layer

### 코드 요약

GraphEmotionNetwork 클래스는 BERT와 그래프 신경망(GNN)을 결합해 감정 분석을 수행하는 모델입니다.

구성 및 동작:

BERT 인코더로 입력 문장을 임베딩(벡터화)합니다.  
각 토큰별 감정 점수(6개 감정)를 추가해 특징을 만듭니다.  
특징을 변환한 뒤, 3개의 Graph Attention Layer(GAT)를 거쳐 토큰 간 관계와 감정 전파를 학습합니다.  
각 감정별로 추가적인 특징 추출 레이어(emotion_propagation)를 적용합니다.  
모든 특징을 합쳐 최종 분류기(classifier)에서 감정(7종: 기쁨, 슬픔, 분노, 공포, 놀람, 혐오, 중립)별 확률을 예측합니다.  
즉, 문장 내 토큰 간의 감정적 연결과 BERT의 언어적 특징을 동시에 활용해, 더 정교한 감정 분석을 목표로 하는 모델입니다.

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from typing import List, Dict, Tuple
# Graph Neural Network 구현 (torch_geometric 없이)

# 한국어 감정 어휘 그래프
EMOTION_GRAPH = {
    'joy': {
        'core': ['기쁘', '행복', '즐겁', '좋'],
        'related': ['웃', '신나', '감동', '사랑', '희망', '만족'],
        'intensifiers': ['매우', '정말', '너무', '아주', '진짜']
    },
    'sadness': {
        'core': ['슬프', '우울', '눈물', '아프'],
        'related': ['외롭', '그립', '후회', '실망', '절망', '비참'],
        'intensifiers': ['너무', '정말', '매우', '몹시']
    },
    'anger': {
        'core': ['화', '분노', '짜증', '싫'],
        'related': ['미워', '열받', '답답', '억울', '불쾌'],
        'intensifiers': ['정말', '너무', '진짜', '완전']
    },
    'fear': {
        'core': ['무서', '두렵', '겁', '공포'],
        'related': ['불안', '걱정', '떨', '긴장', '위험'],
        'intensifiers': ['너무', '정말', '매우']
    },
    'surprise': {
        'core': ['놀라', '깜짝', '충격', '갑작'],
        'related': ['믿기지', '의외', '뜻밖', '예상외'],
        'intensifiers': ['정말', '너무', '완전']
    },
    'disgust': {
        'core': ['역겹', '더럽', '혐오', '구역질'],
        'related': ['불쾌', '끔찍', '징그럽', '메스껍'],
        'intensifiers': ['너무', '정말', '진짜']
    }
}

class EmotionGraphBuilder:
    """텍스트를 감정 그래프로 변환"""

    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.emotion_vocab = self._build_emotion_vocab()

    def _build_emotion_vocab(self):
        """감정 어휘 사전 구축"""
        vocab = {}
        for emotion, words_dict in EMOTION_GRAPH.items():
            for word_list in words_dict.values():
                for word in word_list:
                    if word not in vocab:
                        vocab[word] = []
                    vocab[word].append(emotion)
        return vocab

    def build_graph(self, text):
        """텍스트를 그래프로 변환"""
        tokens = self.tokenizer.tokenize(text)

        # 노드 특징 (감정 초기값)
        node_features = []
        emotion_scores = []

        for token in tokens:
            token_clean = token.replace('##', '')

            # 감정 점수 계산
            scores = np.zeros(6)  # 6개 감정
            for emotion_idx, emotion in enumerate(['joy', 'sadness', 'anger', 'fear', 'surprise', 'disgust']):
                for category, words in EMOTION_GRAPH[emotion].items():
                    for word in words:
                        if word in token_clean:
                            if category == 'core':
                                scores[emotion_idx] += 2.0
                            elif category == 'related':
                                scores[emotion_idx] += 1.0
                            elif category == 'intensifiers':
                                scores[emotion_idx] += 0.5

            emotion_scores.append(scores)

        # 엣지 구성 (인접 토큰 + 의미적 연결)
        edges = []
        edge_weights = []

        for i in range(len(tokens)):
            # 인접 토큰 연결
            if i > 0:
                edges.append([i-1, i])
                edge_weights.append(1.0)
            if i < len(tokens) - 1:
                edges.append([i, i+1])
                edge_weights.append(1.0)

            # 같은 감정 키워드끼리 연결
            for j in range(i+1, min(i+5, len(tokens))):  # 5토큰 이내
                if self._are_emotionally_related(tokens[i], tokens[j]):
                    edges.append([i, j])
                    edges.append([j, i])
                    edge_weights.extend([0.5, 0.5])

        return tokens, np.array(emotion_scores), edges, edge_weights

    def _are_emotionally_related(self, token1, token2):
        """두 토큰이 감정적으로 연관되어 있는지 확인"""
        t1_clean = token1.replace('##', '')
        t2_clean = token2.replace('##', '')

        emotions1 = self.emotion_vocab.get(t1_clean, [])
        emotions2 = self.emotion_vocab.get(t2_clean, [])

        return len(set(emotions1) & set(emotions2)) > 0

class GraphAttentionLayer(nn.Module):
    """Graph Attention Layer 직접 구현"""

    def __init__(self, in_features, out_features, dropout=0.1, alpha=0.2):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.dropout = dropout
        self.alpha = alpha

        self.W = nn.Parameter(torch.zeros(size=(in_features, out_features)))
        nn.init.xavier_uniform_(self.W.data, gain=1.414)

        self.a = nn.Parameter(torch.zeros(size=(2*out_features, 1)))
        nn.init.xavier_uniform_(self.a.data, gain=1.414)

        self.leakyrelu = nn.LeakyReLU(self.alpha)

    def forward(self, x, adj):
        """
        x: [num_nodes, in_features]
        adj: [num_nodes, num_nodes] adjacency matrix
        """
        h = torch.mm(x, self.W)  # [num_nodes, out_features]
        num_nodes = h.size(0)

        # Attention mechanism
        h_repeat = h.repeat(num_nodes, 1)  # [num_nodes*num_nodes, out_features]
        h_repeat_interleave = h.repeat_interleave(num_nodes, dim=0)
        h_concat = torch.cat([h_repeat_interleave, h_repeat], dim=1)  # [num_nodes*num_nodes, 2*out_features]

        e = self.leakyrelu(torch.matmul(h_concat, self.a).squeeze(1))
        e = e.view(num_nodes, num_nodes)

        # Mask attention scores
        zero_vec = -9e15 * torch.ones_like(e)
        attention = torch.where(adj > 0, e, zero_vec)
        attention = F.softmax(attention, dim=1)
        attention = F.dropout(attention, self.dropout, training=self.training)

        h_prime = torch.matmul(attention, h)

        return F.elu(h_prime)

class GraphEmotionNetwork(nn.Module):
    """Graph Neural Network 기반 감정 분석 모델"""

    def __init__(self, model_name='klue/bert-base', hidden_dim=256, num_emotions=7):
        super().__init__()

        # BERT 인코더
        self.bert = AutoModel.from_pretrained(model_name)
        bert_dim = self.bert.config.hidden_size

        # 초기 특징 변환
        self.input_transform = nn.Linear(bert_dim + 6, hidden_dim)  # BERT + 감정 점수

        # Graph Attention Layers
        self.gat1 = GraphAttentionLayer(hidden_dim, hidden_dim, dropout=0.1)
        self.gat2 = GraphAttentionLayer(hidden_dim, hidden_dim, dropout=0.1)
        self.gat3 = GraphAttentionLayer(hidden_dim, hidden_dim, dropout=0.1)

        # 감정 전파 레이어
        self.emotion_propagation = nn.ModuleList([
            nn.Linear(hidden_dim, 64) for _ in range(num_emotions)
        ])

        # 최종 분류기
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim + 64 * num_emotions, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_dim, num_emotions)
        )

        self.dropout = nn.Dropout(0.3)

    def forward(self, input_ids, attention_mask, emotion_scores, edge_index, edge_weight=None):
        # BERT 인코딩
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        bert_features = outputs.last_hidden_state  # [batch, seq_len, bert_dim]

        # 그래프 데이터 준비
        batch_size, seq_len = input_ids.shape
        node_features = []

        for b in range(batch_size):
            # BERT 특징과 감정 점수 결합
            bert_feat = bert_features[b]  # [seq_len, bert_dim]
            emotion_feat = emotion_scores[b] if b < len(emotion_scores) else torch.zeros(seq_len, 6)

            if isinstance(emotion_feat, np.ndarray):
                emotion_feat = torch.tensor(emotion_feat, dtype=torch.float32)

            # 크기 맞추기
            if emotion_feat.shape[0] < seq_len:
                padding = torch.zeros(seq_len - emotion_feat.shape[0], 6)
                emotion_feat = torch.cat([emotion_feat, padding], dim=0)
            elif emotion_feat.shape[0] > seq_len:
                emotion_feat = emotion_feat[:seq_len]

            combined = torch.cat([bert_feat, emotion_feat], dim=-1)
            node_features.append(combined)

        # 노드 특징 변환
        x = torch.stack(node_features).view(-1, bert_features.shape[-1] + 6)
        x = self.input_transform(x)
        x = F.relu(x)

        # Adjacency matrix 생성
        num_nodes = x.shape[0]
        adj = torch.zeros(num_nodes, num_nodes)

        if edge_index is not None and len(edge_index) > 0:
            for edge in edge_index:
                if edge[0] < num_nodes and edge[1] < num_nodes:
                    adj[edge[0], edge[1]] = 1.0
                    adj[edge[1], edge[0]] = 1.0  # 무방향 그래프

        # 자기 연결 추가
        adj = adj + torch.eye(num_nodes)

        # GAT 레이어 통과
        x = self.gat1(x, adj)
        x = self.dropout(x)

        x = self.gat2(x, adj)
        x = self.dropout(x)

        x = self.gat3(x, adj)

        # 감정별 특징 추출
        emotion_features = []
        for emotion_layer in self.emotion_propagation:
            emotion_feat = emotion_layer(x)
            emotion_features.append(emotion_feat)

        # 특징 결합
        combined_features = torch.cat([x] + emotion_features, dim=-1)

        # 최종 분류
        logits = self.classifier(combined_features)

        # Reshape back to [batch, seq_len, num_emotions]
        logits = logits.view(batch_size, seq_len, -1)

        return logits

class GNNEmotionAnalyzer:
    """GNN 기반 감정 분석기"""

    def __init__(self, model, tokenizer, graph_builder):
        self.model = model
        self.tokenizer = tokenizer
        self.graph_builder = graph_builder
        self.emotion_names = ['기쁨', '슬픔', '분노', '공포', '놀람', '혐오', '중립']

    def analyze(self, text):
        """텍스트 감정 분석"""
        # 그래프 구성
        tokens, emotion_scores, edges, edge_weights = self.graph_builder.build_graph(text)

        # 토크나이징
        encoded = self.tokenizer(text, return_tensors='pt', truncation=True, padding=True)

        # 엣지 텐서 변환
        if edges:
            edge_index = torch.tensor(edges, dtype=torch.long)
            edge_weight = torch.tensor(edge_weights, dtype=torch.float)
        else:
            edge_index = torch.tensor([[0], [0]], dtype=torch.long)
            edge_weight = torch.tensor([1.0], dtype=torch.float)

        # 예측
        self.model.eval()
        with torch.no_grad():
            logits = self.model(
                encoded['input_ids'],
                encoded['attention_mask'],
                [emotion_scores],
                edge_index,
                edge_weight
            )

            probs = F.softmax(logits, dim=-1)

        # 결과 정리
        results = []
        valid_tokens = self.tokenizer.convert_ids_to_tokens(encoded['input_ids'][0])

        for i, token in enumerate(valid_tokens):
            if token not in ['[CLS]', '[SEP]', '[PAD]']:
                token_probs = probs[0, i]
                max_prob, max_idx = torch.max(token_probs, dim=0)

                results.append({
                    'token': token,
                    'emotion': self.emotion_names[max_idx.item()],
                    'confidence': max_prob.item(),
                    'all_probs': token_probs.numpy(),
                    'graph_influence': emotion_scores[min(i, len(emotion_scores)-1)] if i < len(emotion_scores) else np.zeros(6)
                })

        return results, edges

    def visualize_emotion_graph(self, text, results, edges):
        """감정 그래프 시각화"""
        G = nx.Graph()

        # 노드 추가
        for i, r in enumerate(results):
            G.add_node(i,
                      label=r['token'],
                      emotion=r['emotion'],
                      confidence=r['confidence'])

        # 엣지 추가
        for edge in edges:
            if edge[0] < len(results) and edge[1] < len(results):
                G.add_edge(edge[0], edge[1])

        # 시각화
        plt.figure(figsize=(14, 8))
        pos = nx.spring_layout(G, k=2, iterations=50)

        # 노드 색상 (감정별)
        emotion_colors = {
            '기쁨': '#FFD700', '슬픔': '#4169E1', '분노': '#DC143C',
            '공포': '#8B008B', '놀람': '#FF69B4', '혐오': '#228B22', '중립': '#C0C0C0'
        }

        node_colors = [emotion_colors[r['emotion']] for r in results]
        node_sizes = [r['confidence'] * 3000 for r in results]

        # 그래프 그리기
        nx.draw_networkx_nodes(G, pos, node_color=node_colors,
                              node_size=node_sizes, alpha=0.7)
        nx.draw_networkx_edges(G, pos, alpha=0.3)

        # 라벨
        labels = {i: r['token'] + '\n' + r['emotion'][:2]
                 for i, r in enumerate(results)}
        nx.draw_networkx_labels(G, pos, labels, font_size=8)

        plt.title(f'감정 그래프: "{text}"')
        plt.axis('off')
        plt.tight_layout()
        plt.show()

def train_gnn_model(model, tokenizer, graph_builder, epochs=20):
    """GNN 모델 간단 훈련"""
    # 훈련 데이터
    train_data = [
        ("정말 기쁜 소식이에요 너무 행복해요", "joy"),
        ("슬픈 이별의 순간이 왔어요", "sadness"),
        ("화가 나서 참을 수가 없어요", "anger"),
        ("무서운 일이 생길까봐 걱정돼요", "fear"),
        ("깜짝 놀랄만한 일이 일어났어요", "surprise"),
        ("역겨운 냄새가 나서 힘들어요", "disgust")
    ]

    optimizer = torch.optim.Adam(model.parameters(), lr=3e-5)
    criterion = nn.CrossEntropyLoss()

    emotion_to_idx = {'joy': 0, 'sadness': 1, 'anger': 2,
                     'fear': 3, 'surprise': 4, 'disgust': 5}

    model.train()
    for epoch in range(epochs):
        total_loss = 0

        for text, emotion in train_data:
            # 그래프 구성
            tokens, emotion_scores, edges, edge_weights = graph_builder.build_graph(text)

            # 토크나이징
            encoded = tokenizer(text, return_tensors='pt', truncation=True,
                              padding='max_length', max_length=128)

            # 레이블 생성
            emotion_idx = emotion_to_idx[emotion]
            labels = torch.full((1, 128), emotion_idx, dtype=torch.long)

            # 엣지 처리
            if edges:
                edge_index = torch.tensor(edges, dtype=torch.long)
            else:
                edge_index = torch.tensor([[0], [0]], dtype=torch.long)

            # 순전파
            optimizer.zero_grad()
            logits = model(
                encoded['input_ids'],
                encoded['attention_mask'],
                [emotion_scores],
                edge_index
            )

            # Loss 계산
            loss = criterion(logits.view(-1, 7), labels.view(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        if (epoch + 1) % 5 == 0:
            print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_data):.4f}')

    return model

def display_gnn_results(text, results):
    """GNN 결과 출력"""
    print(f"\n📝 분석 텍스트: {text}")
    print("\n🔗 그래프 기반 감정 분석:")

    emotion_emoji = {
        '기쁨': '😊', '슬픔': '😢', '분노': '😠',
        '공포': '😨', '놀람': '😲', '혐오': '🤢', '중립': '😐'
    }

    for r in results:
        emoji = emotion_emoji.get(r['emotion'], '')
        graph_influence = r['graph_influence']
        max_influence_idx = np.argmax(graph_influence)
        influence_emotions = ['기쁨', '슬픔', '분노', '공포', '놀람', '혐오']

        if r['emotion'] != '중립':
            print(f"  {r['token']:12s} → {emoji} {r['emotion']:4s} "
                  f"(신뢰도: {r['confidence']:.2%}, "
                  f"그래프 영향: {influence_emotions[max_influence_idx] if graph_influence[max_influence_idx] > 0 else '없음'})")
        else:
            print(f"  {r['token']:12s} → {emoji} {r['emotion']:4s} ({r['confidence']:.2%})")

    # 감정 네트워크 요약
    print("\n🌐 감정 네트워크 특성:")
    emotion_counts = {}
    for r in results:
        if r['emotion'] != '중립':
            emotion_counts[r['emotion']] = emotion_counts.get(r['emotion'], 0) + 1

    if emotion_counts:
        for emotion, count in sorted(emotion_counts.items(), key=lambda x: x[1], reverse=True):
            print(f"  - {emotion}: {count}개 노드가 연결됨")

# 메인 실행
if __name__ == "__main__":
    print("🌐 Graph Neural Network 기반 감정 전파 모델\n")

    # 초기화
    tokenizer = AutoTokenizer.from_pretrained('klue/bert-base')
    graph_builder = EmotionGraphBuilder(tokenizer)
    model = GraphEmotionNetwork('klue/bert-base')
    analyzer = GNNEmotionAnalyzer(model, tokenizer, graph_builder)

    # 간단 훈련
    print("모델 훈련 중...")
    model = train_gnn_model(model, tokenizer, graph_builder, epochs=10)

    # 테스트
    test_texts = [
        "기쁜 소식을 듣고 행복해서 눈물이 났어요",
        "무서운 이야기를 듣고 너무 놀라고 두려웠어요",
        "화가 나서 짜증이 나지만 참고 있어요",
        "놀라운 결과에 기쁘면서도 믿기지 않아요"
    ]

    print("\n" + "="*60)
    print("그래프 기반 감정 분석 결과")
    print("="*60)

    for text in test_texts:
        results, edges = analyzer.analyze(text)
        display_gnn_results(text, results)
        print(f"  → 그래프 엣지 수: {len(edges)}개")
        print("-"*60)

        # 그래프 시각화 (선택적)
        # analyzer.visualize_emotion_graph(text, results, edges)

🌐 Graph Neural Network 기반 감정 전파 모델



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


모델 훈련 중...
Epoch 5/10, Loss: 1.3338
Epoch 10/10, Loss: 0.8100

그래프 기반 감정 분석 결과

📝 분석 텍스트: 기쁜 소식을 듣고 행복해서 눈물이 났어요

🔗 그래프 기반 감정 분석:
  기쁜           → 😊 기쁨   (신뢰도: 25.84%, 그래프 영향: 없음)
  소식           → 😊 기쁨   (신뢰도: 23.48%, 그래프 영향: 없음)
  ##을          → 😊 기쁨   (신뢰도: 21.56%, 그래프 영향: 없음)
  듣            → 😊 기쁨   (신뢰도: 21.11%, 그래프 영향: 없음)
  ##고          → 😊 기쁨   (신뢰도: 22.31%, 그래프 영향: 기쁨)
  행복           → 😊 기쁨   (신뢰도: 22.63%, 그래프 영향: 없음)
  ##해서         → 😠 분노   (신뢰도: 22.66%, 그래프 영향: 슬픔)
  눈물           → 😠 분노   (신뢰도: 23.60%, 그래프 영향: 없음)
  ##이          → 😠 분노   (신뢰도: 24.13%, 그래프 영향: 없음)
  났            → 😠 분노   (신뢰도: 24.41%, 그래프 영향: 없음)
  ##어요         → 😊 기쁨   (신뢰도: 27.92%, 그래프 영향: 없음)

🌐 감정 네트워크 특성:
  - 기쁨: 7개 노드가 연결됨
  - 분노: 4개 노드가 연결됨
  → 그래프 엣지 수: 20개
------------------------------------------------------------

📝 분석 텍스트: 무서운 이야기를 듣고 너무 놀라고 두려웠어요

🔗 그래프 기반 감정 분석:
  무서운          → 😨 공포   (신뢰도: 30.19%, 그래프 영향: 없음)
  이야기          → 😨 공포   (신뢰도: 28.93%, 그래프 영향: 없음)
  ##를          → 😨 공포   (신뢰도: 26.51